In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications import vgg16
from keras.applications import mobilenet
from tensorflow.keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_path  = '/content/drive/MyDrive/dl_data/0724/train'
valid_path  = '/content/drive/MyDrive/dl_data/0724/valid'
test_path  = '/content/drive/MyDrive/dl_data/0724/test'

In [4]:
train_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(
    train_path, target_size=(224,224), batch_size=30)
valid_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(
    valid_path, target_size=(224,224), batch_size=30)
test_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(
    test_path, target_size=(224,224), batch_size=30)

Found 202 images belonging to 2 classes.
Found 103 images belonging to 2 classes.
Found 451 images belonging to 2 classes.


In [5]:
base_model = vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (224,224, 3))
base_model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [6]:
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model

In [7]:
last_layer = base_model.get_layer('block5_pool')

In [8]:
last_output = last_layer.output

In [9]:
x = Flatten()(last_output)

In [10]:
x = Dense(64, activation='relu', name='FC_2')(x)
x = BatchNormalization()(x) # 배치 단위로 데이터를 정규화 -> 안정적인 학습
x = Dropout(0.5)(x) # 과적합을 줄이기 위함

In [11]:
x = Dense(2, activation='softmax', name='softmax')(x)
# 개/고양이 분류기

In [12]:
new_model = Model(inputs=base_model.input, outputs=x)

In [13]:
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [14]:
new_model.compile(Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
new_model.fit_generator(train_batches, steps_per_epoch=4,
                   validation_data=valid_batches, validation_steps=2, epochs=20, verbose=2)

<ipython-input-15-6c0f84f949e8>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  new_model.fit_generator(train_batches, steps_per_epoch=4,


Epoch 1/20
4/4 - 77s - loss: 1.1574 - accuracy: 0.5357 - val_loss: 2.0405 - val_accuracy: 0.7167 - 77s/epoch - 19s/step
Epoch 2/20
4/4 - 9s - loss: 0.7295 - accuracy: 0.6750 - val_loss: 1.1946 - val_accuracy: 0.7833 - 9s/epoch - 2s/step
Epoch 3/20
4/4 - 3s - loss: 0.4436 - accuracy: 0.7946 - val_loss: 2.6442 - val_accuracy: 0.7000 - 3s/epoch - 768ms/step
Epoch 4/20
4/4 - 2s - loss: 0.3224 - accuracy: 0.8667 - val_loss: 1.6732 - val_accuracy: 0.7500 - 2s/epoch - 585ms/step
Epoch 5/20
4/4 - 2s - loss: 0.2976 - accuracy: 0.8917 - val_loss: 3.9569 - val_accuracy: 0.7167 - 2s/epoch - 616ms/step
Epoch 6/20
4/4 - 3s - loss: 0.2913 - accuracy: 0.8833 - val_loss: 2.5170 - val_accuracy: 0.7167 - 3s/epoch - 654ms/step
Epoch 7/20
4/4 - 2s - loss: 0.2112 - accuracy: 0.8929 - val_loss: 1.3044 - val_accuracy: 0.8167 - 2s/epoch - 583ms/step
Epoch 8/20
4/4 - 2s - loss: 0.2078 - accuracy: 0.9083 - val_loss: 1.5756 - val_accuracy: 0.7833 - 2s/epoch - 624ms/step
Epoch 9/20
4/4 - 2s - loss: 0.3014 - accura

In [16]:
def load_dataset(path):
    data = load_files(path)
    paths = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']))
    return paths, targets

In [19]:
from sklearn.datasets import load_files
# from keras.utils import np_utils
# 에러 발생시
# from tf.keras.utils import np_utils
# from tensorflow.keras.utils import np_utils
import numpy as np
test_files, test_targets = load_dataset('/content/drive/MyDrive/dl_data/0724/test')

ModuleNotFoundError: No module named 'tf'

In [ ]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from tqdm import tqdm
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)
def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)
test_tensors = preprocess_input(paths_to_tensor(test_files))

In [ ]:
print('\nTesting loss: {:.4f}\nTesting accuracy: {:.4f}'.format(*new_model.evaluate(test_tensors, test_targets)))

In [ ]:
# evaluate and print test accuracy
score = new_model.evaluate(test_tensors, test_targets)
print('\n', 'Test accuracy:', score[1])